# MODIS land cover exploration

Visualisation and stats generation

In [3]:
import ee

In [4]:
ee.Initialize()

In [5]:
import ipyleaflet

# utility
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

MODIS land cover data, `ee.ImageColelction` with 17 `ee.Image`, each representing a year (data between 2000 and 2016). Each year image has multiple bands, representing different land cover classifications

In [6]:
modis = ee.ImageCollection('MODIS/006/MCD12Q1')

In [7]:
meta_modis = modis.getInfo()

In [8]:
def get_modis_lc_by_year(year):
    
    if year not in range(2000, 2017):
        raise Exception('Year must be in between 2000 and 2016')
        
    # internal ee.Image 'system:time_start' 'system:time_end' in miliseconds
    g = modis.filterDate(str(year)+'-01-01', str(year)+'-12-31').first()
    
    return g

In [9]:
lc2015 = get_modis_lc_by_year(2015)

In [10]:
lc2015

Here `g` is an instance of `ee.Image`. Get an descriptive object `meta` to check on properties. Examine if the time filter works as intended. `meta` will also be used to derive colour schemes and etc. 

**N.B.**
For viz purposes, `meta` is the same regardless of the band/different LC classification within `modis`. This can be re-used and there is no need to repeatedly request `meta` from Earth Engine, based on the band

In [11]:
meta = lc2015.getInfo()

In [12]:
meta['properties']['system:time_start']

1420070400000

In [13]:
import datetime

In [14]:
datetime.datetime.fromtimestamp(meta['properties']['system:time_start']/1000.0)

datetime.datetime(2015, 1, 1, 0, 0)

In [15]:
datetime.datetime.fromtimestamp(meta['properties']['system:time_end']/1000.0)

datetime.datetime(2016, 1, 1, 0, 0)

Here are the properties of the `meta`, which is a dictionary

In [16]:
meta['properties'].keys()

[u'system:index',
 u'LC_Prop1_class_values',
 u'system:time_start',
 u'LC_Type3_class_values',
 u'LC_Type4_class_palette',
 u'system:footprint',
 u'LC_Prop2_class_names',
 u'LC_Type4_class_names',
 u'LW_class_palette',
 u'LC_Type5_class_values',
 u'LC_Prop2_class_values',
 u'LC_Type4_class_values',
 u'LC_Type5_class_names',
 u'LC_Type1_class_values',
 u'LC_Type2_class_palette',
 u'LC_Prop1_class_names',
 u'LC_Type1_class_names',
 u'LW_class_values',
 u'system:time_end',
 u'LC_Type3_class_palette',
 u'LC_Prop3_class_names',
 u'LC_Type2_class_values',
 u'LC_Prop3_class_palette',
 u'LC_Type2_class_names',
 u'LC_Prop1_class_palette',
 u'LC_Prop3_class_values',
 u'LC_Type3_class_names',
 u'system:asset_size',
 u'LC_Type1_class_palette',
 u'LC_Type5_class_palette',
 u'LW_class_names',
 u'LC_Prop2_class_palette']

## Visualisation

In [17]:
import ipyleaflet

# utility
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [19]:
# function to return viz params, to be unpacked for use
def viz_setting(band):
    '''e.g. band="LC_Type1" '''
    values = meta['properties'][band+'_class_values']
    palette = meta['properties'][band+'_class_palette']
    
    return {'min': min(values),
           'max': max(values),
           'palette': palette,
           'bands': [band]}

# convenient wrapper
def viz_map(ee_image, band):
    map1 = ipyleaflet.Map(
    center=(48.2082, 16.3779), zoom=4,
    layout={'height':'400px'}
    )

    map1.add_layer(
        ipyleaflet.TileLayer(url=GetTileLayerUrl(
            # returns ee_image object
            ee_image.visualize(**viz_setting(band))
        )
    ))

    return map1

In [20]:
from IPython.core.display import display, HTML

# legend viz
def viz_legend(band):
    '''Create a legend based on band'''
    values = meta['properties'][band+'_class_values']
    palette = meta['properties'][band+'_class_palette']
    names = meta['properties'][band+'_class_names']
    
    html = ""
    for value, name, colour in zip(values, names, palette):
        html += '''
        <p style="font-size:12px">
        <div style="background-color:#{};display:inline-block;width:30px;height:12px">
        </div> {}: {}</p>
        '''.format(colour, value, name)
        
#     print html
    display(HTML(html))

## Examples

Below are examples of all LC classification within `modis` in year 2015

In [21]:
# description of the modis
display(HTML(meta_modis['properties']['description']))

Name,Units,Min,Max,Description
LC_Type1,,,,Land Cover Type 1: Annual International Geosphere-Biosphere Programme (IGBP) classification
LC_Type2,,,,Land Cover Type 2: Annual University of Maryland (UMD) classification
LC_Type3,,,,Land Cover Type 3: Annual Leaf Area Index (LAI) classification
LC_Type4,,,,Land Cover Type 4: Annual BIOME-Biogeochemical Cycles (BGC) classification
LC_Type5,,,,Land Cover Type 5: Annual Plant Functional Types classification
LC_Prop1_Assessment,%,0,100,LCCS1 land cover layer confidence
LC_Prop2_Assessment,%,0,100,LCCS2 land use layer confidence
LC_Prop3_Assessment,%,0,100,LCCS3 surface hydrology layer confidence
LC_Prop1,,,,FAO-Land Cover Classification System 1 (LCCS1) land cover layer
LC_Prop2,,,,FAO-LCCS2 land use layer


In [22]:
# Land Cover Type 1: Annual International Geosphere-Biosphere Programme (IGBP) classification
viz_map(lc2015, 'LC_Type1')

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [23]:
viz_legend('LC_Type1')

In [24]:
# Land Cover Type 2: Annual University of Maryland (UMD) classification
viz_map(lc2015, 'LC_Type2')

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [25]:
viz_legend('LC_Type2')

In [26]:
# Land Cover Type 3: Annual Leaf Area Index (LAI) classification
viz_map(lc2015, 'LC_Type3')

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [27]:
viz_legend('LC_Type3')

## Zonal statistics

Use a geometry to:
1. cut the image using a mask
2. derive some statistics about the region of innterest


In [114]:
testmap = viz_map(ee_image=lc2015, band='LC_Type1')

In [29]:
testmap

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [30]:
type(testmap)

ipyleaflet.leaflet.Map

## Get geojson from PPNET

PP API token for testing purpose

In [33]:
ppapi_token = '4290b88825725a4d241c485d3b0b7cd7'

In [82]:
from ipyleaflet import GeoJSON, Map

In [117]:
import urllib2 as urllib
import json

# construct for get request
def get_ppnet_url(wdpaid, token):
    url = "https://api.protectedplanet.net/v3/protected_areas/{}?token={}".format(wdpaid, token)
    return url

# load url and get wdpa data
def get_pa(url):
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    
    return data

# only to get geometry in geojson
def get_geojson(wdpaid, token):
    url = get_ppnet_url(wdpaid, token)
    data = get_pa(url)
    
    if 'protected_area' not in data.keys():
        raise Exception('no protected area in json')
    else:
        data = data['protected_area']
        
        if 'geojson' not in data.keys():
            raise Exception('no geometry in json')
        else:
            return data['geojson']
        

In [118]:
get_ppnet_url(2008, ppapi_token)

'https://api.protectedplanet.net/v3/protected_areas/2008?token=4290b88825725a4d241c485d3b0b7cd7'

In [271]:
# get_geojson(2008, ppapi_token)

Test of overlaying geojson

wdpaid 2008 is a World Heritage site in between Poland and Belarus

In [109]:
m = Map(center=(52.72491,23.9697784), zoom=9)

In [110]:
geojson = GeoJSON(data=get_geojson(2008, ppapi_token), style={})

In [59]:
type(geojson)

ipyleaflet.leaflet.GeoJSON

In [111]:
m.add_layer(geojson)

In [112]:
m

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


In [116]:
testmap.add_layer(geojson)
testmap.center = [52.72491,23.9697784]
testmap.zoom = 9
testmap

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


## Stats exploring

In [128]:
# construct ee.Feature
def get_eefeature(wdpaid, token):
    pa = get_pa(url=get_ppnet_url(wdpaid, token))
    
    pa = pa['protected_area']
    
    # construct ee.Feature geom + dict of attr
    return ee.Feature(pa['geojson'], pa.pop('geojson'))


Using bialo as example to check `ee.Geometry` and `ee.Feature` methods

In [131]:
bialo = get_eefeature(2008, ppapi_token)

In [134]:
geom = bialo.geometry()

In [136]:
geom.projection().getInfo()

{u'crs': u'EPSG:4326',
 u'transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 u'type': u'Projection'}

Very confusing coordinate systems in GEE. I cannot understand what's happening behind the blackbox

In [145]:
wgs84 = ee.Projection('EPSG:4326')
mwd = ee.Projection('''PROJCS["World_Mollweide",
    GEOGCS["GCS_WGS_1984",
        DATUM["WGS_1984",
            SPHEROID["WGS_1984",6378137,298.257223563]],
        PRIMEM["Greenwich",0],
        UNIT["Degree",0.017453292519943295]],
    PROJECTION["Mollweide"],
    PARAMETER["False_Easting",0],
    PARAMETER["False_Northing",0],
    PARAMETER["Central_Meridian",0],
    UNIT["Meter",1],
    AUTHORITY["EPSG","54009"]]''')

In [141]:
ee.Projection('EPSG:4326').getInfo()

{u'crs': u'EPSG:4326',
 u'transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 u'type': u'Projection'}

In [144]:
ee.Projection('''PROJCS["World_Mollweide",
    GEOGCS["GCS_WGS_1984",
        DATUM["WGS_1984",
            SPHEROID["WGS_1984",6378137,298.257223563]],
        PRIMEM["Greenwich",0],
        UNIT["Degree",0.017453292519943295]],
    PROJECTION["Mollweide"],
    PARAMETER["False_Easting",0],
    PARAMETER["False_Northing",0],
    PARAMETER["Central_Meridian",0],
    UNIT["Meter",1],
    AUTHORITY["EPSG","54009"]]''').getInfo()

{u'transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 u'type': u'Projection',
 u'wkt': u'PROJCS["World_Mollweide", \n  GEOGCS["GCS_WGS_1984", \n    DATUM["WGS_1984", \n      SPHEROID["WGS_1984", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["central_meridian", 0.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH], \n  AUTHORITY["EPSG","54009"]]'}

In [248]:
bialo.area(maxError=0.1, proj=mwd).getInfo()

1391490381.9833984

According to GEE, `ee.Feature.area` returns the area of the feature's default geomtry in square meters

In [158]:
# I'm not sure how it is done?
# it is quite a significant difference compared to reported areas
# in square km
bialo.area().getInfo() / (1000.0 * 1000.0)

1020132630.3024315

In [159]:
# I'm not sure how it is done?
# it is quite a significant difference compared to reported areas
# in square km
bialo.area().getInfo()

1020132630302431.5

In [153]:
bialo_pp = get_pa(get_ppnet_url(2008,ppapi_token))

In [156]:
bialo_pp['protected_area']['reported_area']

u'1418.85'

In [164]:
bialo.geometry().area().getInfo()

1020132630302431.5

In [171]:
geom.projection().getInfo()

{u'crs': u'EPSG:4326',
 u'transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 u'type': u'Projection'}

Some pretty big differences - could it be it uses the default web mercator in the calculation

Another example of very simple geometry: Galapagos Island

In [172]:
gala = get_eefeature(191, ppapi_token)

In [175]:
gala_pp = get_pa(get_ppnet_url(191, ppapi_token))

In [173]:
geom2 = gala.geometry()

In [174]:
geom2.projection().getInfo()

{u'crs': u'EPSG:4326',
 u'transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 u'type': u'Projection'}

In [179]:
gala.area().getInfo() / 10000000.0

50991888.30823038

In [251]:
# reported area from PPNET
gala_pp['protected_area']['reported_area']

u'140665.14'

In [270]:
# use degree - this obviously is silly but just checking...
gala.area(maxError=1, proj=ee.Projection('EPSG:4326')).getInfo()

11.869987367663974

In [252]:
# use molleweide
gala.area(maxError=0.1, proj=mwd).getInfo()

147067257322.15942

In [253]:
# use molleweide but different maxerror
gala.area(maxError=0.01, proj=mwd).getInfo()

147067256767.8296

Convert to raster and display - check if `ee.Feature` had been constructed correctly

In [181]:
gala_r = ee.Image().byte()

In [204]:
gala_image = gala_r.paint(ee.FeatureCollection(gala), 
                         color=1,
                         width=3)

Try `reducer` on image to obtain area - this looks consistent

In [238]:
gala_area = ee.Image.pixelArea()

In [242]:
gala_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=gala.geometry(),
    scale=150).getInfo()

{u'area': 146083820175.86685}

In [243]:
gala_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=gala.geometry(),
    scale=350).getInfo()

{u'area': 146083100492.89636}

Looks like `geojson` from PPAPI is cast correctly into `ee.Feature`

In [217]:
map1 = ipyleaflet.Map(layout={'height':'400px'}, zoom=6, center=(-0.4734209,-90.8904806))

In [218]:
map1.add_layer(
    ipyleaflet.TileLayer(url=GetTileLayerUrl(gala_image))

)

In [219]:
map1

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


## Zonal statistics - land cover

Grouped reductions. Statistics in each zone of an `image` by using reducer.group()

Construct an `ee.Image` of two bands, a `pixelArea` band (used to sum total in area) and a `land cover` band (used to provide the group definition)

In [254]:
landcover = ee.Image.pixelArea().addBands(lc2015.select('LC_Type1'))

In [257]:
landcover.bandNames().getInfo()

[u'area', u'LC_Type1']

Use `bialo` as input geom, and `landcover` input image. `groupField` is the index of the band containing the zones by which to group the output, in this case, `groupField=1` refers to the `landcover` band (The `area` band has index 0)

In [260]:
result = landcover.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='landcover'
    ),
    geometry=bialo.geometry(),
    scale=100
).getInfo()

In [265]:
result['groups']

[{u'landcover': 1, u'sum': 222177846.67470837},
 {u'landcover': 4, u'sum': 30881579.170831423},
 {u'landcover': 5, u'sum': 1058747306.4780327},
 {u'landcover': 8, u'sum': 27617364.872995164},
 {u'landcover': 9, u'sum': 31727085.910864748},
 {u'landcover': 10, u'sum': 22154713.868397675},
 {u'landcover': 11, u'sum': 364804.9365234375},
 {u'landcover': 12, u'sum': 39981.42554955574},
 {u'landcover': 13, u'sum': 268159.6290690104}]

In [268]:
import numpy as np

Validation to check the sum of `sum` of each land cover type is the same as the total area of the site. In this case, it is very close to 1400 sqkm, the same as from the `rep_area` from the WDPA via PPNET API

In [269]:
np.array([each['sum'] for each in result['groups']]).sum()

1393978842.9669724

Sensitivity analysis to check GEE's variable `scale` and `maxPixels`

In [282]:
def landcover_composition(ee_landcover, ee_feature, scale):
    # `ee.Image`, `ee.Feature`, scale
    # compose a two band image 0: area 1: land cover classes
    landcover = ee.Image.pixelArea().addBands(ee_landcover)
    
    # reduce group
    result = landcover.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='landcover'
    ),
    geometry=ee_feature.geometry(),
    scale=scale,
    maxPixels=1e9
    ).getInfo()
    
    return result['groups']

In [283]:
landcover_composition(ee_landcover=lc2015.select('LC_Type1'), ee_feature=bialo, scale=10)

[{u'landcover': 1, u'sum': 223332400.53329718},
 {u'landcover': 4, u'sum': 30749167.52696558},
 {u'landcover': 5, u'sum': 1057534493.666615},
 {u'landcover': 8, u'sum': 27760601.240934633},
 {u'landcover': 9, u'sum': 31759449.05298227},
 {u'landcover': 10, u'sum': 22101869.32274595},
 {u'landcover': 11, u'sum': 436672.8308868408},
 {u'landcover': 12, u'sum': 34940.76837234496},
 {u'landcover': 13, u'sum': 254108.40444635172}]

In [274]:
landcover_composition(ee_landcover=lc2015.select('LC_Type1'), ee_feature=bialo, scale=100)

[{u'landcover': 1, u'sum': 222177846.67470837},
 {u'landcover': 4, u'sum': 30881579.170831423},
 {u'landcover': 5, u'sum': 1058747306.4780327},
 {u'landcover': 8, u'sum': 27617364.872995164},
 {u'landcover': 9, u'sum': 31727085.910864748},
 {u'landcover': 10, u'sum': 22154713.868397675},
 {u'landcover': 11, u'sum': 364804.9365234375},
 {u'landcover': 12, u'sum': 39981.42554955574},
 {u'landcover': 13, u'sum': 268159.6290690104}]

In [275]:
landcover_composition(ee_landcover=lc2015.select('LC_Type1'), ee_feature=bialo, scale=1000)

[{u'landcover': 1, u'sum': 210743367.60269606},
 {u'landcover': 4, u'sum': 27402440.68063725},
 {u'landcover': 5, u'sum': 1061467705.3438724},
 {u'landcover': 8, u'sum': 34027615.602696076},
 {u'landcover': 9, u'sum': 35080003.93259804},
 {u'landcover': 10, u'sum': 24808394.12328432},
 {u'landcover': 12, u'sum': 162202.9705882353},
 {u'landcover': 13, u'sum': 264253.3470588235}]

In [276]:
landcover_composition(ee_landcover=lc2015.select('LC_Type1'), ee_feature=bialo, scale=10000)

[{u'landcover': 1, u'sum': 1908987.607843137},
 {u'landcover': 5, u'sum': 1312704721.1607845},
 {u'landcover': 10, u'sum': 29204153.74117647},
 {u'landcover': 12, u'sum': 58535499.27843137}]

This illustrate the effect of scale in GEE. Image pyramids are used and depending on scale, values will be draw from the image pyramids, i.e. not the original values. This is different and misleading composed to other GIS and image processing platforms.

**IMPORTANT**
For discrete valued images, pixel values of upper levels of the pyramids are a sample of the **top left pixel of the next lower level**. It is not unimaginable that a signifcant number of values may disappear entirely when values are drawn from top most levels of the pyramids.

[How scale works in Google Earth Engine](https://developers.google.com/earth-engine/scale)

## Clipping images

In [285]:
lc2015.clip(bialo)

In [287]:
clipped_map = viz_map(ee_image=lc2015.clip(bialo), band='LC_Type1')
clipped_map.center = [52.72491,23.9697784]
clipped_map.zoom = 9
clipped_map

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=
